In [2]:
import numpy as np
import math
import sys
from scipy import special
from scipy import linalg

In [5]:
class primitive_gaussian():
    
    def __init__(self, alpha, coeff, coordinates, l1, l2, l3):
    
        self.alpha = alpha
        self.coeff = coeff
        self. coordinates = np.array(coordinates)
        self.l1 = l1
        self.l2 = l2
        self.l3 = l3
        # Normalisation constant
        self.A = (2.0* alpha / np.pi) ** 0.75  # + other terms when we have l1, l2 ,l3 (for p orbital, d orbital)
        

In [6]:
def boys(x, n):
    if x==0:
        return 1.0/(2*n+1)
    
    else:
        return special.gammainc(n+0.5, x) * special.gamma(n+0.5) * (1.0/(2*x**(n+0.5)))

In [22]:
# Note this integral is only for s-orbital systems, therefore no angular momentum terms

def electron_electron_repulsion(molecule):
    
    nbasis = len(molecule)
    
    V_ee = np.zeros([nbasis, nbasis, nbasis, nbasis])
    
    # e-e integral repulsion matrix is 4-dimensional matrix
    
    for i in range(nbasis):
        for j in range(nbasis):
            for k in range(nbasis):
                for l in range(nbasis):
                    
                    nprimitives_i = len(molecule[i])
                    nprimitives_j = len(molecule[j])
                    nprimitives_k = len(molecule[k])
                    nprimitives_l = len(molecule[l])
                    
                    for ii in range(nprimitives_i):
                        for jj in range(nprimitives_j):
                            for kk in range(nprimitives_k):
                                for ll in range(nprimitives_l):
                                    
                                    N = molecule[i][ii].A * molecule[j][jj].A * molecule[k][kk].A * molecule[l][ll].A
                                    cicjckcl = molecule[i][ii].coeff * molecule[j][jj].coeff * molecule[k][kk].coeff * molecule[l][ll].coeff
                                    
                                    pij = molecule[i][ii].alpha + molecule[j][jj].alpha
                                    pkl = molecule[k][kk].alpha + molecule[l][ll].alpha
                                    
                                    Pij = molecule[i][ii].alpha*molecule[i][ii].coordinates + molecule[j][jj].alpha*molecule[j][jj].coordinates 
                                    Pkl = molecule[k][kk].alpha*molecule[k][kk].coordinates + molecule[l][ll].alpha*molecule[l][ll].coordinates 
                                    
                                    Ppij = Pij/pij
                                    Ppkl = Pkl/pkl
                                    
                                    PpijPpkl = Ppij - Ppkl
                                    PpijPpkl2 = np.dot(PpijPpkl , PpijPpkl)
                                    denom = 1.0/pij + 1.0/pkl
                                    
                                    qij = molecule[i][ii].alpha * molecule[j][jj].alpha / pij
                                    qkl = molecule[k][kk].alpha * molecule[l][ll].alpha / pkl
                                    
                                    Qij = molecule[i][ii].coordinates - molecule[j][jj].coordinates
                                    Qkl = molecule[k][kk].coordinates - molecule[l][ll].coordinates
                                    
                                    Q2ij = np.dot(Qij, Qij)
                                    Q2kl = np.dot(Qkl, Qkl)
                                    
                                    term1 = 2.0*math.pi*math.pi/(pij*pkl)
                                    term2 = math.sqrt( math.pi/(pij+pkl))
                                    term3 = math.exp(-qij*Q2ij)
                                    term4 = math.exp(-qkl*Q2kl)
                                    
                                    V_ee[i,j,k,l] += N* cicjckcl *term1 * term2 * term3 * term4 * boys(PpijPpkl2/denom,0) # 3 more
                                    
    return V_ee

In [23]:
# STO-3G basis for 1s orbital on hydrogen
# These are descriptions of atomic orbital basis 
                            # Alpha,           # coeff            # Coord   # angular momentum
H1_pg1a = primitive_gaussian (0.3425250914E+01, 0.1543289673E+00, [0,0,0], 0, 0, 0)
H1_pg1b = primitive_gaussian (0.6239137298E+00 , 0.5353281423E+00, [0,0,0], 0, 0, 0)
H1_pg1c = primitive_gaussian (0.1688554040E+00, 0.4446345422E+00, [0,0,0], 0, 0, 0)

H2_pg1a = primitive_gaussian (0.3425250914E+01, 0.1543289673E+00, [1.4,0,0], 0, 0, 0)
H2_pg1b = primitive_gaussian (0.6239137298E+00 , 0.5353281423E+00, [1.4,0,0], 0, 0, 0)
H2_pg1c = primitive_gaussian (0.1688554040E+00, 0.4446345422E+00, [1.4,0,0], 0, 0, 0)

H1_1s = [H1_pg1a, H1_pg1b, H1_pg1c]  # Describing the atomic orbital with slater function
H2_1s = [H2_pg1a, H2_pg1b, H2_pg1c]

Z = [1.0, 1.0]
atom_coordinates = [np.array([0, 0, 0]), np.array([1.4, 0, 0])]
molecule = [H1_1s, H2_1s]

In [24]:
e_e_repulsion = electron_electron_repulsion(molecule)

In [25]:
print(e_e_repulsion)

[[[[0.77460594 0.44410766]
   [0.44410766 0.56967593]]

  [[0.44410766 0.29702854]
   [0.29702854 0.44410766]]]


 [[[0.44410766 0.29702854]
   [0.29702854 0.44410766]]

  [[0.56967593 0.44410766]
   [0.44410766 0.77460594]]]]
